# Logistic Regression Assignment - Complete Demonstration

This notebook demonstrates the complete implementation of Logistic Regression from scratch.

## Assignment Overview
- **Problem 1**: Implement hypothesis function (sigmoid)
- **Problem 2**: Implement gradient descent
- **Problem 3**: Implement prediction methods
- **Problem 4**: Implement loss function with regularization
- **Problem 5**: Train and evaluate on iris dataset
- **Problem 6**: Plot learning curves
- **Problem 7**: Visualize decision boundaries
- **Problem 8**: Save/load weights


In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from scratch_logistic_regression import ScratchLogisticRegression

print("All libraries imported successfully!")


## 1. Load and Prepare Data

We'll use the Iris dataset and perform binary classification between Versicolor (class 1) and Virginica (class 2).


In [ ]:
# Load iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Binary classification: versicolor (1) vs virginica (2)
mask = (y == 1) | (y == 2)
X = X[mask]
y = y[mask]

# Convert labels to 0 and 1
y = (y == 2).astype(int)

print(f"Dataset shape: {X.shape}")
print(f"Number of samples: {len(y)}")
print(f"Class distribution: {np.bincount(y)}")
print(f"Feature names: {iris.feature_names}")


In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train_scaled.shape}")
print(f"Test set: {X_test_scaled.shape}")


## 2. Problem 5: Train Scratch Implementation

Train our custom logistic regression implementation.


In [ ]:
# Split training data for validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Initialize and train scratch model
scratch_model = ScratchLogisticRegression(
    num_iter=1000,
    lr=0.1,
    bias=True,
    verbose=True
)

scratch_model.fit(X_train_split, y_train_split, X_val_split, y_val_split, lambda_reg=0.01)

print("\nModel training complete!")


In [ ]:
# Evaluate scratch model
y_pred_scratch = scratch_model.predict(X_test_scaled)
y_proba_scratch = scratch_model.predict_proba(X_test_scaled)

print("Scratch Model Performance:")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_scratch):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_scratch):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_scratch):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_scratch))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_scratch, target_names=['Versicolor', 'Virginica']))


## 3. Compare with Scikit-learn Implementation


In [ ]:
# Train scikit-learn model
sklearn_model = LogisticRegression(max_iter=1000, random_state=42)
sklearn_model.fit(X_train_scaled, y_train)

# Evaluate sklearn model
y_pred_sklearn = sklearn_model.predict(X_test_scaled)

print("Scikit-learn Model Performance:")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_sklearn):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_sklearn):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_sklearn):.4f}")

# Compare
print("\n" + "="*50)
print("Comparison:")
print(f"Accuracy difference: {abs(accuracy_score(y_test, y_pred_scratch) - accuracy_score(y_test, y_pred_sklearn)):.4f}")


## 4. Problem 6: Plot Learning Curve


In [ ]:
# Plot learning curve
plt.figure(figsize=(10, 6))
plt.plot(range(scratch_model.iter), scratch_model.loss, label='Training Loss', linewidth=2)
plt.plot(range(scratch_model.iter), scratch_model.val_loss, label='Validation Loss', linewidth=2)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Learning Curve: Loss vs Iteration', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final training loss: {scratch_model.loss[-1]:.6f}")
print(f"Final validation loss: {scratch_model.val_loss[-1]:.6f}")


## 5. Problem 7: Visualize Decision Boundary

We'll train a model using only 2 features (Sepal Width and Petal Length) for 2D visualization.


In [ ]:
# Prepare 2D dataset (using sepal width and petal length)
X_2d = X[:, [1, 2]]
X_train_2d, X_test_2d, y_train_2d, y_test_2d = train_test_split(
    X_2d, y, test_size=0.3, random_state=42, stratify=y
)

# Standardize
scaler_2d = StandardScaler()
X_train_2d_scaled = scaler_2d.fit_transform(X_train_2d)
X_test_2d_scaled = scaler_2d.transform(X_test_2d)

# Split for validation
X_train_2d_split, X_val_2d_split, y_train_2d_split, y_val_2d_split = train_test_split(
    X_train_2d_scaled, y_train_2d, test_size=0.2, random_state=42, stratify=y_train_2d
)

# Train 2D model
model_2d = ScratchLogisticRegression(
    num_iter=1000,
    lr=0.1,
    bias=True,
    verbose=False
)

model_2d.fit(X_train_2d_split, y_train_2d_split, X_val_2d_split, y_val_2d_split)

print("2D model trained successfully!")


In [ ]:
# Plot decision boundary
def plot_decision_boundary(model, X, y):
    # Create mesh grid
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Predict for each point in the mesh
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.4, cmap='RdYlBu')
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, s=100, edgecolors='black', 
                         linewidth=1.5, cmap='RdYlBu', alpha=0.8)
    plt.xlabel('Sepal Width (standardized)', fontsize=12)
    plt.ylabel('Petal Length (standardized)', fontsize=12)
    plt.title('Decision Boundary Visualization', fontsize=14, fontweight='bold')
    plt.colorbar(scatter, label='Class')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Combine train and test for visualization
X_all_2d = np.vstack([X_train_2d_scaled, X_test_2d_scaled])
y_all_2d = np.hstack([y_train_2d, y_test_2d])

plot_decision_boundary(model_2d, X_all_2d, y_all_2d)


## 6. Problem 8: Save and Load Weights


In [ ]:
# Save weights using np.savez
scratch_model.save_weights('models/model_weights')

# Save complete model using pickle
scratch_model.save_model('models/model_complete.pkl')

print("\nOriginal model coefficients (first 3):")
print(scratch_model.coef_[:3])


In [ ]:
# Load weights
new_model = ScratchLogisticRegression(num_iter=1000, lr=0.1, bias=True, verbose=False)
new_model.load_weights('models/model_weights.npz')

print("\nLoaded model coefficients (first 3):")
print(new_model.coef_[:3])

print(f"\nCoefficients match: {np.allclose(scratch_model.coef_, new_model.coef_)}")


In [ ]:
# Load complete model
loaded_model = ScratchLogisticRegression.load_model('models/model_complete.pkl')

# Test loaded model
y_pred_loaded = loaded_model.predict(X_test_scaled)
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)

print(f"Loaded model accuracy: {accuracy_loaded:.4f}")
print(f"Original model accuracy: {accuracy_score(y_test, y_pred_scratch):.4f}")
print(f"\nModels produce identical predictions: {np.array_equal(y_pred_scratch, y_pred_loaded)}")


In [ ]:
# Display learned coefficients
print("Learned Coefficients:")
print(f"Bias term: {scratch_model.coef_[0]:.4f}")
print("\nFeature coefficients:")
for i, name in enumerate(iris.feature_names):
    print(f"  {name}: {scratch_model.coef_[i+1]:.4f}")

# Visualize coefficients
plt.figure(figsize=(10, 6))
plt.bar(range(len(iris.feature_names)), scratch_model.coef_[1:])
plt.xlabel('Features', fontsize=12)
plt.ylabel('Coefficient Value', fontsize=12)
plt.title('Feature Coefficients', fontsize=14, fontweight='bold')
plt.xticks(range(len(iris.feature_names)), iris.feature_names, rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()


## 8. Summary

This notebook has successfully demonstrated:

✅ **Problem 1**: Implemented sigmoid hypothesis function  
✅ **Problem 2**: Implemented gradient descent with regularization  
✅ **Problem 3**: Implemented predict() and predict_proba() methods  
✅ **Problem 4**: Implemented loss function with regularization  
✅ **Problem 5**: Trained and evaluated on iris dataset, compared with sklearn  
✅ **Problem 6**: Plotted learning curves  
✅ **Problem 7**: Visualized decision boundaries  
✅ **Problem 8**: Demonstrated weight saving and loading  

The scratch implementation performs comparably to scikit-learn's implementation!
